In [6]:
import multiprocessing as mp
import netket as nk
import os
import json
from qutip import *
import numpy as np
import scipy
import cmath
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import itertools
from numpy.random import seed
from numpy.random import rand
from itertools import product
from scipy.misc import derivative
from scipy import optimize
from functools import wraps
from time import time
from scipy.optimize import minimize
from matplotlib import gridspec

In [7]:
def CSHam(N, B, Ak):
    # Make graph with of length N with no periodic boundary conditions
    g = nk.graph.Hypercube(length=N, n_dim=1, pbc=False)
    # Spin based Hilbert Space
    hilbertSpace = nk.hilbert.Spin(s=0.5, N=g.n_nodes)
    # Define spin operators with \hbar set to 1
    sz = 0.5 * np.array([[1, 0], [0, -1]])
    sx = 0.5 * np.array([[0, 1], [1, 0]])
    sy = 0.5 * np.array([[0, -1j], [1j, 0]])
    operators = []
    sites = []
    # Central spin term
    operators.append((B * sz).tolist()) #array to list(ordered and changeable)
    sites.append([0])
    # Interaction term
    itOp = np.kron(sz, sz) + np.kron(sx, sx) + np.kron(sy, sy) #kronecker product here
    for i in range(N - 1):
        operators.append((Ak[i] * itOp).tolist())
        sites.append([0, (i+1)])  #pretty convoluted indexing, but ok
    # Create hamiltonian
    hamiltonian = nk.operator.LocalOperator(hilbertSpace, operators=operators, acting_on=sites, dtype=complex)
    #acting_on specifier necessary as this is a central spin model
    return hamiltonian, hilbertSpace

def CSHam_ex(N, B, Ak, GS, Filling):
    # Make graph with of length N with no periodic boundary conditions
    g = nk.graph.Hypercube(length=N, n_dim=1, pbc=False)
    # Spin based Hilbert Space
    hilbertSpace = nk.hilbert.Spin(s=0.5, N=g.n_nodes)
    # Define spin operators with \hbar set to 1
    sz = 0.5 * np.array([[1, 0], [0, -1]])
    sx = 0.5 * np.array([[0, 1], [1, 0]])
    sy = 0.5 * np.array([[0, -1j], [1j, 0]])
    operators = []
    sites = []
    # Central spin term
    operators.append((B * sz).tolist()) #array to list(ordered and changeable)
    sites.append([0])
    # Interaction term
    itOp = np.kron(sz, sz) + np.kron(sx, sx) + np.kron(sy, sy) #kronecker product here
    for i in range(N - 1):
        operators.append((Ak[i] * itOp).tolist())
        sites.append([0, (i+1)])  #pretty convoluted indexing, but ok
    # now we add the extra gap filler
    #GS_s = scipy.sparse.csr_matrix(GS)
    #fill_Op = Filling * scipy.sparse.kron(GS_s, scipy.sparse.spmatrix.getH(GS_s))
    fill_Op = Filling * np.outer(GS, np.conj(GS))
    operators.append(fill_Op)
    sites.append(np.arange(0,N).tolist())
    # Create hamiltonian
    hamiltonian = nk.operator.LocalOperator(hilbertSpace, operators=operators, acting_on=sites, dtype=complex)
    #acting_on specifier necessary as this is a central spin model
    return hamiltonian, hilbertSpace

def exactDiagonalization(hamiltonian):
    # Changes Hamiltonian to matrix form, where hamiltonian of interest is sparse in matrix form
    #haMatrix = hamiltonian.to_sparse()
    # Gets eigenvalues and vectors, where the built-in function uses 
    eigenValues, v = nk.exact.lanczos_ed(hamiltonian, compute_eigenvectors=True)

    # Orders from smallest to largest
    eigenVectors = [v[:, i] for i in range(len(eigenValues))]
    return eigenValues, eigenVectors

def exactDiagonalization_full(hamiltonian):
    # Changes Hamiltonian to matrix form
    haMatrix = hamiltonian.to_dense()
    # Gets eigenvalues and vectors
    eigenValues, v = np.linalg.eigh(haMatrix)
    # Orders from smallest to largest
    eigenVectors = [v[:, i] for i in range(len(eigenValues))]
    return eigenValues, eigenVectors

# Error Calculation (Input: the found state, the state from exact diagonalization, the found energy, the energy from exact diagonalization)
def err(state, edState, eng, edEng,N):
    engErr = np.abs(eng - edEng)
    overlap = np.dot(state.conj().reshape(2**N, 1).T, edState.reshape(2**N, 1))
    waveFunctionErr = 1 - (np.linalg.norm(overlap))**2
    return engErr, waveFunctionErr

In [1]:
import os
os.chdir('/home/victor/Dynamics')
os.getcwd()

'/home/victor/Dynamics'

In [33]:
def normalize(v):
    norm = np.linalg.norm(v)
    return v / norm


N=2
B=0.95
Ak = []
for i in range(N-1):
        # Constant A
    Ak_i = 1
        # Variable A
        #Ak_i = A / (N0) * np.exp(-i / N0)
    Ak.append(Ak_i)

ha, hi = CSHam(N, B, Ak)
psi0 = normalize(np.array([1,2,2,50]))
driver = nk.exact.PyExactTimePropagation(ha, t0=0, dt=1, initial_state=psi0, propagation_type="real")

for step in driver.iter(n_iter=300):
    print(driver.estimate(ha))

-2.231e-01+0.000e+00j ± 0.000e+00 [σ²=1.795e-03]
-2.231e-01+0.000e+00j ± 0.000e+00 [σ²=1.795e-03]
-2.231e-01+0.000e+00j ± 0.000e+00 [σ²=1.795e-03]
-2.231e-01+0.000e+00j ± 0.000e+00 [σ²=1.795e-03]
-2.231e-01+0.000e+00j ± 0.000e+00 [σ²=1.795e-03]
-2.231e-01+0.000e+00j ± 0.000e+00 [σ²=1.795e-03]
-2.231e-01+0.000e+00j ± 0.000e+00 [σ²=1.795e-03]
-2.231e-01+0.000e+00j ± 0.000e+00 [σ²=1.795e-03]
-2.231e-01+0.000e+00j ± 0.000e+00 [σ²=1.795e-03]
-2.231e-01+0.000e+00j ± 0.000e+00 [σ²=1.795e-03]
-2.231e-01+0.000e+00j ± 0.000e+00 [σ²=1.795e-03]
-2.231e-01+0.000e+00j ± 0.000e+00 [σ²=1.795e-03]
-2.231e-01+0.000e+00j ± 0.000e+00 [σ²=1.795e-03]
-2.231e-01+0.000e+00j ± 0.000e+00 [σ²=1.795e-03]
-2.231e-01+0.000e+00j ± 0.000e+00 [σ²=1.795e-03]
-2.231e-01+0.000e+00j ± 0.000e+00 [σ²=1.795e-03]
-2.231e-01+0.000e+00j ± 0.000e+00 [σ²=1.795e-03]
-2.231e-01+0.000e+00j ± 0.000e+00 [σ²=1.795e-03]
-2.231e-01+0.000e+00j ± 0.000e+00 [σ²=1.795e-03]
-2.231e-01+0.000e+00j ± 0.000e+00 [σ²=1.795e-03]
-2.231e-01+0.000e+00